# Closures
## Capturing Variables
In most languages with closures, garbage collection plays an important role. For example, consider this JavaScript code:
```Rust
// Start an animation that rearranges the rows in a table of cities.
function startSortingAnimation(cities, stat) {
    // Helper function that we'll use to sort the table. 
    // Note that this function refers to stat.
    function keyfn(city) {
        return city.get_statistic(stat); 
    }
    if (pendingSort) pendingSort.cancel();
    // Now kick off an animation, passing keyfn to it. 
    // The sorting algorithm will call keyfn later. 
    pendingSort = new SortingAnimation(cities, keyfn);
}
```
The closure `keyfn` is stored in the new `SortingAnimation` object. It’s meant to be called after `startSortingAnimation` returns. Now, normally when a function returns, all its variables and arguments go out of scope and are discarded. But here, the JavaScript engine must keep `stat` around somehow, since the closure uses it. Most JavaScript engines do this by allocating stat in the heap and letting the garbage collector reclaim it later.

Rust doesn’t have garbage collection. How will this work? To answer this question, we’ll look at two examples.

### Closures That Borrow
```Rust
/// Sort by any of several different statistics.
fn sort_by_statistic(cities: &mut Vec<City>, stat: Statistic) { 
    cities.sort_by_key(|city| -city.get_statistic(stat));
}
```

实际上，这个例子有问题，sort_by_key接受一个FnMut的参数，能执行例子中的代码的前提是，Statistic实现了Copy trait，书本中并未说明。

In this case, when Rust creates the closure, it automatically borrows a reference to stat. It stands to reason: the closure refers to stat, so it must have a reference to it.
The rest is simple. The closure is subject to the rules about borrowing and lifetimes that we described in **Chapter 5**. In particular, since the closure contains a reference to stat, Rust won’t let it outlive stat. Since the closure is only used during sorting, this example is fine.

In short, Rust ensures safety by using lifetimes instead of garbage collection. Rust’s way is faster: even a fast GC allocation will be slower than storing stat on the stack, as Rust does in this case.

来看看别的语言是如何处理闭包的：
```javascript
// Start an animation that rearranges the rows in a table of cities.
function startSortingAnimation(cities, stat) {
    // Helper function that we'll use to sort the table. 
    // Note that this function refers to stat.
    function keyfn(city) {
        return city.get_statistic(stat); 
    }
    if (pendingSort) 
        pendingSort.cancel();
    // Now kick off an animation, passing keyfn to it. 
    // The sorting algorithm will call keyfn later. 
    pendingSort = new SortingAnimation(cities, keyfn);
}
```

在这里，闭包存储在了`new SortingAnimation`这个对象中。一般来说，离开函数作用域后，变量就会被GC回收。但是此处，为了保全这个闭包，js会在堆上开辟空间让这个变量苟活，使得这个变量稍后再被GC回收。


## Function and Closure Types
```Rust
/// Sort by any of several different statistics.
fn sort_by_statistic(cities: &mut Vec<City>, stat: Statistic) { 
    cities.sort_by_key(|city| -city.get_statistic(stat));
}
```
紧接着上面这个例子，其实 任何type `fn(&City) -> i64` 都可以作为 `sort_by_key` 的参数。这是因为："In addition, all safe function pointers implement Fn, FnMut, and FnOnce, because these traits are specially known to the compiler." —— https://doc.rust-lang.org/std/primitive.fn.html

所以，以下实现也是没有问题的：

```Rust
fn city_population_descending(city: &City) -> i64 { 
    -city.population
}

cities.sort_by_key(|city| -city.get_statistic(stat));
```

In [4]:
struct City {
    name: String,
    population: i64,
    country: String,
}

fn city_population_descending(city: &City) -> i64 { 
    -city.population
}

impl City {
    fn new(name: &str, population: i64, country: &str) -> Self {
        Self {
            name: name.to_string(),
            population,
            country: country.to_string(),
        }
    }
}

let mut cities = vec![
    City::new("Dubai", 3_137_000, "United Arab Emirates"),
    City::new("Hangzhou", 6_242_000, "China"),
    City::new("Lima", 10_069_000, "Peru"),
    City::new("Sofia", 1_204_000, "Bulgaria"),
    City::new("Calgary", 1_285_000, "Canada"),
    City::new("Rome", 2_627_000, "Italy"),
    City::new("Cape Town", 3_740_000, "South Africa"),
    City::new("Moscow", 12_537_000, "Russia"),
    City::new("Shanghai", 34_000_000, "China"),
    City::new("Tokyo", 38_050_000, "Japan"),
];

cities.sort_by_key(city_population_descending);

for city in cities {
    println!("{:20} {:>12}k", city.name, city.population / 1000);
};

Tokyo                       38050k
Shanghai                    34000k
Moscow                      12537k
Lima                        10069k
Hangzhou                     6242k
Cape Town                    3740k
Dubai                        3137k
Rome                         2627k
Calgary                      1285k
Sofia                        1204k


**Closure is callable, but closure is not fn**.

In fact, every closure you write has its own type, because a closure may contain data: values either borrowed or stolen from enclosing scopes. This could be any number of variables, in any combination of types. So every closure has an ad hoc type created by the compiler, large enough to hold that data. No two closures have exactly the same type. But every closure implements an Fn trait.

Since every closure has its own type, code that works with closures usually needs to be generic.